In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [47]:
import numpy as np
from collections import defaultdict

import scipy.spatial as spa

from Bio.PDB import get_surface, PDBParser, ShrakeRupley, PDBIO, Select
from Bio.PDB.PDBExceptions import PDBConstructionWarning

from bigbind import get_rec_coords, PocketSelect

In [98]:
class Pocket:
    
    def __init__(self):
        self.coords = []
        self.radii = []
        
    def add_sphere(self, x, y, z, radius):
        self.coords.append(np.array([x, y, z]))
        self.radii.append(radius)
        
    def get_residues(self, res_indexes, res_coords, dist_cutoff):
        mask = np.zeros(len(res_coords), dtype=bool)
        for coord, radius in zip(self.coords, self.radii):
            mask |= spa.distance.cdist(res_coords, np.array([coord])).min(axis=1) < radius + dist_cutoff
        included = set(res_indexes[mask])
        return included

def get_pockets(pqr_fname):
    pocket_dict = defaultdict(Pocket)
    pqr = open(pqr_fname).read()
    for line in pqr.splitlines():
        if line.startswith("ATOM"):
            _, _,_, _, idx, x, y, z, _, r = line.split()
            pocket_dict[int(idx)].add_sphere(float(x), float(y), float(z), float(r))
    return pocket_dict
pocket_dict = get_pockets("/home/boris/Data/BigBind/P06401/4apu_B_rec_out/4apu_B_rec_pockets.pqr")
# pocket_dict[1].get_residues(res_indexes, coords)

In [39]:
biopython_parser = PDBParser()
rec_file = "/home/boris/Data/BigBind/P06401/4apu_B_rec.pdb"
structure = biopython_parser.get_structure('random_id', rec_file)
rec = structure[0]

In [97]:
out_folder = "/home/boris/Data/BigBind/P06401/4apu_B_rec_out/"
res_indexes, coords = get_rec_coords(rec)
for idx, pocket in pocket_dict.items():
    included = pocket.get_residues(res_indexes, coords, 0)
    io = PDBIO()
    io.set_structure(structure)
    out_file = f"{out_folder}/pocket_{idx}.pdb"
    io.save(out_file, PocketSelect(included))
    print(idx, len(included))

1 26
2 15
3 6
4 7
5 8
6 10
7 10
8 8
9 10
10 5
11 9
12 8
13 4
14 5
15 11
